In [61]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. what for
1. 获取MDN的输出

# 1.

In [71]:
import random
import pickle
import time

import torch.utils.data
import torchvision.transforms as transforms
import visdom

from MLP.mydataset import *
import torch.nn as nn
import MLP.my_collate_fn
from MLP.my_collate_fn import my_collate_fn_3
from torch.utils.data import DataLoader, SequentialSampler

from importlib import reload
from torch.nn.utils import clip_grad_norm_

from MLP.models import Conv_1_4

######### Ray Tune

import MLP.loss
import MLP.plot

reload(MLP.loss)    # 必须reload！！

reload(MLP.my_collate_fn)

from MLP.Config.config_revenue import DefaultConfig
reload(MLP.Config.config_revenue)
opt = DefaultConfig()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
from MLP.loss import validate


In [72]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(opt.seed)

dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.data_key_path, opt.NLL_metric_path)


data_idx = np.arange(0,dataset.__len__()).tolist()
len_all = len(data_idx)
print("seed=",opt.seed)

seed= 31


In [73]:
# 使用全部的data
shuffled_indices = np.random.permutation(len_all)

In [74]:
data_loader = DataLoader(dataset=dataset, batch_size=opt.batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SequentialSampler(data_idx), collate_fn=my_collate_fn_3)

In [75]:
mlp = Conv_1_4(opt.N_gaussians)
model_data = torch.load(opt.net_root_path) #  map_location=device
mlp.load_state_dict(model_data)
mlp = mlp.to(device=device)

- 按照pi,scale,shape的顺序存储，和MDN的输出顺序一致

In [76]:
# pi,scale,shape
output_MDN = np.array([0.]*opt.N_gaussians*3*len_all).reshape(-1,opt.N_gaussians*3)
output_MDN_pd = pd.DataFrame(output_MDN)
output_MDN_pd.shape

(1276, 6)

In [77]:
model=mlp.eval()
for batch_id, data in enumerate(data_loader):

    input_data, _,_, _, _ = data
    # Do the inference
    input_data = input_data.to(device)

    pi,scale,shape = mlp(input_data)

    pi = pi.detach().cpu().numpy().squeeze()
    scale = scale.detach().cpu().numpy().squeeze()
    shape = shape.detach().cpu().numpy().squeeze()

    for j in range(len(pi)):

        output_MDN_pd.iloc[batch_id,0+j*3] = pi[j]
        output_MDN_pd.iloc[batch_id,1+j*3] = scale[j]
        output_MDN_pd.iloc[batch_id,2+j*3] = shape[j]

print("Done")
print(output_MDN_pd.head())

Done
          0           1         2         3           4         5
0  0.118766  208.873566  1.064327  0.881234  650.974121  1.420142
1  0.281571  249.948166  0.924029  0.718429  532.396057  1.265382
2  0.169522  384.012085  1.215992  0.830478  989.968994  1.397663
3  0.735808   84.600212  0.897451  0.264192   86.613136  0.976443
4  0.895222   86.128151  1.011115  0.104778  135.253555  1.546015


In [78]:
opt.params_save_path

'../data/revenue/output_MDN_seed=31.csv'

In [80]:
output_MDN_pd.to_csv(opt.params_save_path,encoding="utf-8",header=True,index=False)

# 2. 对testset
1. 先读取bid voucher的id

In [18]:
data_setting = pd.read_csv(opt.data_key_path, encoding="utf-8")

In [195]:
voucher_idx = []
for i in range(data_setting.shape[0]):
    if "Voucher" in data_setting.loc[i,'desc']:
        voucher_idx.append(i)
    elif "VOUCHER" in data_setting.loc[i,'desc']:
        voucher_idx.append(i)
voucher_idx.__len__()

40

In [29]:
if opt.SET_VAL:
    train_idx = shuffled_indices[:int(opt.train_pct * len_all)]
    tmp = int((opt.train_pct + opt.vali_pct) * len_all)
    val_idx = shuffled_indices[int(opt.train_pct * len_all):tmp]
    test_idx = shuffled_indices[tmp:]
else:
    train_idx = shuffled_indices[:int(opt.train_pct * len_all)]
    tmp = int((opt.train_pct + opt.vali_pct) * len_all)
    test_idx = shuffled_indices[int(opt.train_pct * len_all):tmp]
    val_idx = shuffled_indices[tmp:]

- dataloader

In [30]:
if opt.USE_VOUCHER: # 算上voucher,一般情况都会算的
    test_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(test_idx), collate_fn=my_collate_fn_3)

# 不算voucher
else:
    test_idx_wo_voucher = [idx for idx in test_idx if idx not in voucher_idx]
    test_loader = DataLoader(dataset=dataset, batch_size=opt.batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SequentialSampler(test_idx_wo_voucher), collate_fn=my_collate_fn_3)

- 计算metric

In [31]:
model=mlp.eval()
with torch.no_grad():

    total_test_metric, GT_metric,odds = validate(mlp, test_loader, opt.N_gaussians, opt.MIN_LOSS, device,detail = True)

    GT_metric_1 = GT_metric[0, 0]# 和GT-1的差值：越大越好
    metric_diff = GT_metric_1 - total_test_metric
    GT_1 = GT_metric_1
    print(f"USE_VOUCHER = {opt.USE_VOUCHER}")
    print(f"GT_metric = {GT_metric}, NN prediction = {total_test_metric}")
    print(f"better than GT1 = {metric_diff}")
    print(f"the GTs: GT1,GT2(common),GT2(SA), GT2(NN)")
    print(f"The odds:  {odds}")
    print(f"seed = {opt.seed}")


USE_VOUCHER = True
GT_metric = tensor([[6.8396, 9.0867, 8.0615, 6.8613]]), NN prediction = 6.303638935089111
better than GT1 = 0.5360097885131836
the GTs: GT1,GT2(common),GT2(SA), GT2(NN)
The odds:  [0.5411764705882353, 0.7333333333333333, 0.5215686274509804]
seed = 223
